In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import networkx as nx

from interfaces.db import DB

from investment.concept_configuration import normalise

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
db = DB()

## Fetch data from database

In [5]:
# Get EPFLStartups
table_name = 'graph_piper.Nodes_N_EPFLStartup'
fields = ['EPFLStartupID', 'StartupName']
conditions = {'Status': 'Private'}
startups = pd.DataFrame(db.find(table_name, fields=fields, conditions=conditions), columns=fields)

In [6]:
# Get startups' founders
table_name = 'graph_piper.Edges_N_EPFLStartup_N_Person_T_Founder'
fields = ['EPFLStartupID', 'SCIPER']
startups_founders = pd.DataFrame(db.find(table_name, fields=fields), columns=fields)
founder_ids = list(startups_founders['SCIPER'].drop_duplicates())

In [7]:
# Get startups' professors
table_name = 'graph_piper.Edges_N_EPFLStartup_N_Person_T_Professor'
fields = ['EPFLStartupID', 'SCIPER']
startups_professors = pd.DataFrame(db.find(table_name, fields=fields), columns=fields)
professor_ids = list(startups_professors['SCIPER'].drop_duplicates())

In [8]:
# Get founders' concepts
table_name = 'graph_piper.Edges_N_Person_N_Concept_T_Research'
fields = ['SCIPER', 'PageID', 'Score']
conditions = {'SCIPER': founder_ids}
founders_concepts = pd.DataFrame(db.find(table_name, fields=fields, conditions=conditions), columns=fields)
founders_concept_ids = list(founders_concepts['PageID'].drop_duplicates())

In [9]:
# Get professors' concepts
table_name = 'graph_piper.Edges_N_Person_N_Concept_T_Research'
fields = ['SCIPER', 'PageID', 'Score']
conditions = {'SCIPER': professor_ids}
professors_concepts = pd.DataFrame(db.find(table_name, fields=fields, conditions=conditions), columns=fields)
professors_concept_ids = list(professors_concepts['PageID'].drop_duplicates())

In [10]:
# Get concepts
table_name = 'graph_piper.Nodes_N_Concept_T_Title'
fields = ['PageID', 'PageTitle']
conditions = {'PageID': list(set(founders_concept_ids + professors_concept_ids))}
concepts = pd.DataFrame(db.find(table_name, fields=fields, conditions=conditions), columns=fields)

## Merge tables to obtain EPFLStartup-Concept edges

In [11]:
startups_founders = pd.merge(startups, startups_founders, how='inner', on='EPFLStartupID')
startups_founders_concepts = pd.merge(startups_founders, founders_concepts, how='inner', on='SCIPER')
startups_founders_concepts = startups_founders_concepts.groupby(by=['EPFLStartupID', 'PageID']).aggregate({'Score': 'sum'}).reset_index()
startups_founders_concepts = pd.merge(startups, startups_founders_concepts, how='inner', on='EPFLStartupID')

In [12]:
startups_professors = pd.merge(startups, startups_professors, how='inner', on='EPFLStartupID')
startups_professors_concepts = pd.merge(startups_professors, professors_concepts, how='inner', on='SCIPER')
startups_professors_concepts = startups_professors_concepts.groupby(by=['EPFLStartupID', 'PageID']).aggregate({'Score': 'sum'}).reset_index()
startups_professors_concepts = pd.merge(startups, startups_professors_concepts, how='inner', on='EPFLStartupID')

In [13]:
# We give priority to the founders' concepts and use the professors' concepts as fallback
startup_with_founder_ids = list(startups_founders_concepts['EPFLStartupID'].drop_duplicates())
startups_professors_concepts = startups_professors_concepts[~startups_professors_concepts['EPFLStartupID'].isin(startup_with_founder_ids)]

In [14]:
startups_concepts = pd.concat([startups_founders_concepts, startups_professors_concepts]).reset_index(drop=True)

In [15]:
startups_concepts = normalise(startups_concepts)

In [16]:
startup_ids = list(startups_concepts['EPFLStartupID'].drop_duplicates())
startups = startups[startups['EPFLStartupID'].isin(startup_ids)].reset_index(drop=True)
startups = startups.rename(columns={'EPFLStartupID': 'ID', 'StartupName': 'Name'})
startups['Type'] = 'Startup'

In [17]:
concept_ids = list(startups_concepts['PageID'].drop_duplicates())
concepts = concepts[concepts['PageID'].isin(concept_ids)].reset_index(drop=True)
concepts = concepts.rename(columns={'PageID': 'ID', 'PageTitle': 'Name'})
concepts['Type'] = 'Concept'

## Transform data into lists of nodes and edges

In [18]:
nodes_df = pd.concat([startups, concepts]).reset_index(drop=True)
nodes = [(row['ID'], {'name': row['Name'], 'type': row['Type']}) for row in nodes_df.to_dict(orient='records')]

In [19]:
edges_df = startups_concepts.rename(columns={'EPFLStartupID': 'SourceID', 'StartupName': 'Name', 'PageID': 'TargetID', 'Score': 'Weight'})[['SourceID', 'TargetID', 'Weight']]
edges = [(row['SourceID'], row['TargetID'], {'weight': row['Weight']}) for row in edges_df.to_dict(orient='records')]

## Create networkx graph and export it as .gexf

In [20]:
G = nx.Graph()

G.add_nodes_from(nodes)
G.add_edges_from(edges)

nx.write_gexf(G, "startups.gexf")

## (Check all ids in edges show up in nodes)

In [21]:
aaa = pd.merge(edges_df, nodes_df, how='left', left_on='TargetID', right_on='ID')
aaa[aaa['ID'].isna()]

,SourceID,TargetID,Weight,ID,Name,Type
1276,es-im4tb,22681820,0.019311,NaN,NaN,NaN
1707,es-lightbend,56150,0.017371,NaN,NaN,NaN
1816,es-lightbend,39765053,0.037753,NaN,NaN,NaN
2047,es-metaphysiks-engineering,2089136,0.008793,NaN,NaN,NaN
2116,es-metaphysiks-engineering,28903680,0.008395,NaN,NaN,NaN
2891,es-odoma,16290254,0.080870,NaN,NaN,NaN
3478,es-phase-solutions,11022628,0.012993,NaN,NaN,NaN
3721,es-regenosca,317082,0.101197,NaN,NaN,NaN
3786,es-rheon-medical,231444,0.067408,NaN,NaN,NaN
3928,es-rheon-medical,37601885,0.004990,NaN,NaN,NaN
